# This notebook calculates the hazard mitigation funding metric sourced from FEMA
* Total amount of hazard mitigation funding recieved per CA county between 1990-2023

In [1]:
import pandas as pd
import os
import sys
import math

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/governance/community_preparedness/fema/hazard_mitigation_funding/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'hazard_mitigation_funding_subset.csv'


In [3]:
# read in FEMA emergency performance grants data
hazard_mitigation_data = pd.read_csv('hazard_mitigation_funding_subset.csv')
print(len(hazard_mitigation_data))
os.remove('hazard_mitigation_funding_subset.csv')

2131


In [12]:
hazard_mitigation_data.columns

Index(['projectIdentifier', 'programArea', 'programFy', 'region', 'state',
       'stateNumberCode', 'county', 'countyCode', 'disasterNumber',
       'projectCounties', 'projectType', 'status', 'recipient',
       'recipientTribalIndicator', 'subrecipient',
       'subrecipientTribalIndicator', 'dataSource', 'dateApproved',
       'dateClosed', 'dateInitiallyApproved', 'projectAmount',
       'federalShareObligated', 'subrecipientAdminCostAmt', 'srmcObligatedAmt',
       'recipientAdminCostAmt', 'costSharePercentage', 'benefitCostRatio',
       'netValueBenefits', 'numberOfFinalProperties', 'numberOfProperties',
       'id'],
      dtype='object')

In [4]:
# renaming and selecting relevant columns to our CRI metric
hazard_mitigation_columns = hazard_mitigation_data[['programFy', 'county', 'projectAmount']].copy()

hazard_mitigation_columns.rename(columns={'programFy': 'funding_year', 
                                      'projectAmount': 'project_amount'}, inplace=True)

hazard_mitigation_columns

,funding_year,county,project_amount
0,2019,Ventura,2159446.00
1,2022,Santa Barbara,196315.00
2,2018,Marin,1385685.00
3,2019,Ventura,1083095.00
4,2011,San Luis Obispo,3334701.39
...,...,...,...
2126,1997,Sacramento,3164945.00
2127,1992,San Bernardino,1108678.00
2128,1993,San Bernardino,170000.00
2129,1993,San Bernardino,100000.00


In [5]:
# running the filter county function to get rid of any non-CA county entries
filtered_hazard_mitigation, omitted_rows = filter_counties(hazard_mitigation_columns, 'county')

In [6]:
# Group the DataFrame by 'county' and 'year', then sum the 'damaged_destroyed_structures' and 'total_fatalities'
summed_hazard_mitigation = filtered_hazard_mitigation.groupby(['county', 'funding_year']).agg({'project_amount': 'sum'}).reset_index()

print(len(summed_hazard_mitigation))
summed_hazard_mitigation.head()

504


,county,funding_year,project_amount
0,Alameda,1990,43439324.0
1,Alameda,1992,5097938.0
2,Alameda,1993,2904070.0
3,Alameda,1994,57804311.0
4,Alameda,2003,382740.0


#### Non-CA county entries are 'statewide' and nan

In [7]:
unique_counties = omitted_rows['county'].unique()
print(unique_counties)

['Statewide' nan]


In [8]:
# Display unique years
unique_years = summed_hazard_mitigation['funding_year'].unique()
print(unique_years)

[1990 1992 1993 1994 2003 2004 2005 2006 2007 2008 2009 2010 2015 2017
 2018 2020 2021 2022 2023 2014 1995 1997 2019 2011 1998 2002 2012 1999
 2000 2016 2013]


In [9]:
# Group the data further by just county and sum the funding recieved for each county from 1990-2023
cri_hazard_mitigation_total_county = summed_hazard_mitigation.groupby(['county']).agg({'project_amount': 'sum'}).reset_index()
cri_hazard_mitigation_total_county.rename(columns={'project_amount': 'funding_1990_2023'}, inplace=True)
pd.set_option('display.float_format', '{:.2f}'.format)

print(len(cri_hazard_mitigation_total_county))
cri_hazard_mitigation_total_county.head()

58


,county,funding_1990_2023
0,Alameda,193501555.81
1,Alpine,319033.91
2,Amador,8581918.04
3,Butte,22708739.59
4,Calaveras,10750389.00


In [10]:
# Saving metric df to .csv file
cri_hazard_mitigation_total_county.to_csv('governance_hazard_mitigation_metric.csv')

In [11]:
# upload final csv file to aws
bucket_name = 'ca-climate-index'
file_name = ['governance_hazard_mitigation_metric.csv']
directory = '3_fair_data/index_data'

#@append_metadata
upload_csv_aws(file_name, bucket_name, directory)
os.remove('governance_hazard_mitigation_metric.csv')

governance_hazard_mitigation_metric.csv uploaded to AWS


### Function call for this metric

In [10]:
@append_metadata
def calc_emergency_management_funding(df, export=False, export_filename=None, varname = ''):
    '''
    Calculates the total amount of hazard mitigation funding received per CA county between 
    1990-2023 from FEMA: 
    https://www.fema.gov/openfema-data-page/hazard-mitigation-assistance-projects-v3

    Methods
    -------
    Data was cleaned to isolate funding for California counties. 
    Funding from 1990-2023 was summed per county.

    Parameters
    ----------
    df: string
        the dataframe containing the initial hazard mitigation funding data
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI hazard mitigation funding metric to AWS
        True = will upload resulting df containing CAL CRAI emergency hazard mitigation funding metric to AWS
    export_filename: string
        name of the csv file to be uploaded to AWS

    Script
    ------
    governance_hazard_mitigation_funding.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    
    '''

    print('Data transformation: selecting and renaming relevant columns.')
    # renaming and selecting relevant columns to our CRI metric
    hazard_mitigation_columns = hazard_mitigation_data[['programFy', 'county', 'projectAmount']].copy()

    hazard_mitigation_columns.rename(columns={'programFy': 'funding_year', 
                                        'projectAmount': 'project_amount'}, inplace=True)

    print('Data transformation: apply "filter_counties" to county column to isolate CA counties.')
    print('Data transformation: dropping all columns with recipients labeled "statewide" due to lack of clarity as to distribution at county/census tract scale.')
    # running the filter county function to get rid of any non-CA county entries
    filtered_hazard_mitigation, omitted_rows = filter_counties(hazard_mitigation_columns, 'county')

    print('Data transformation: group the data by county and sum funding from 1990-2023.')
    # Group the DataFrame by 'county' and 'year', then sum the 'damaged_destroyed_structures' and 'total_fatalities'
    summed_hazard_mitigation = filtered_hazard_mitigation.groupby(['county', 'funding_year']).agg({'project_amount': 'sum'}).reset_index()

    # Group the data further by just county and sum the funding recieved for each county from 1990-2023
    cri_hazard_mitigation_total_county = summed_hazard_mitigation.groupby(['county']).agg({'project_amount': 'sum'}).reset_index()
    cri_hazard_mitigation_total_county.rename(columns={'project_amount': 'funding_1990_2023'}, inplace=True)

    # export to csv and upload to AWS
    if export == True:
        cri_hazard_mitigation_total_county.to_csv(export_filename)
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [export_filename]
        upload_csv_aws(export_filename, bucket_name, directory)

        # Check if the file exists before attempting to remove it
        if os.path.exists('hazard_mitigation_funding_subset.csv'):
            os.remove('hazard_mitigation_funding_subset.csv')  # remove from local to clear up directory

        if os.path.exists(export_filename[0]):
            os.remove(export_filename[0])

    return cri_hazard_mitigation_total_county # returns df

In [11]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/governance/community_preparedness/fema/hazard_mitigation_funding/'
pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

hazard_mitigation_data = pd.read_csv('hazard_mitigation_funding_subset.csv')

calc_emergency_management_funding(hazard_mitigation_data, export=True, export_filename = 'governance_hazard_mitigation_metric.csv', varname='governance_fema_hazard_mitigation')

Saved DataFrame as 'hazard_mitigation_funding_subset.csv'


,county,funding_1990_2023
0,Alameda,193501555.81
1,Alpine,319033.91
2,Amador,8581918.04
3,Butte,22708739.59
4,Calaveras,10750389.00
5,Colusa,704934.00
6,Contra Costa,29297026.24
7,Del Norte,6852908.00
8,El Dorado,15237169.06
9,Fresno,976567.12
